# Librerias

In [1]:
# Carga de librerías
import numpy as np
import tensorflow as tf
import PIL
import PIL.Image
import pathlib
import scipy as sp
import seaborn as sns
from itertools import product
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import os
import pathlib
from pathlib import Path  

In [2]:
from numpy.linalg import LinAlgError
from scipy.sparse.csgraph import laplacian
# from spectral_metric.lib import find_samples, compute_expectation_with_monter_carlo 
from spectral_metric.estimator import CumulativeGradientEstimator
from spectral_metric.visualize import make_graph

# DATASET DESDE DIRECTORIO 

In [3]:
directorio_dataset = ('../Datasets/CIFAR-10 dos clases')
imagenes_originales = ('../Datasets/CIFAR-10 dos clases/cat')
imagenes_corruptas = ('../Datasets/CIFAR-10 dos clases/cat_c_defocus_blur')

directorio_dataset = pathlib.Path(directorio_dataset)
imagenes_originales = pathlib.Path(imagenes_originales)
imagenes_corruptas = pathlib.Path(imagenes_corruptas)

total_dataset = len(list(directorio_dataset.glob('*/*.png')))  # Cuenta la cantidad de imagenes del dataset (Aqui busca en las subcarpetas del directorio del dataset)
total_originales = len(list(imagenes_originales.glob('*.png')))  # Cuenta la cantidad de imagenes de la clase original (Aqui busca directamente en la carpeta cat)
total_corruptas = len(list(imagenes_corruptas.glob('*.png')))  # Cuenta la cantidad de imagenes de la clase corrupta (Aqui busca directamente en la carpeta cat_c_defocus_blur)

print(f"Total imagenes dataset: {total_dataset}")
print(f"Total imagenes originales: {total_originales}")
print(f"Total imagenes corruptas: {total_corruptas}")

Total imagenes dataset: 10000
Total imagenes originales: 5000
Total imagenes corruptas: 5000


In [4]:
# Dimensiones de imagen y tamaño de batch
img_height = 32
img_width = 32
#batch_size = len(list(data_dir.glob('*/*.png'))) # leer todas las imágenes al tiempo
batch_size = total_dataset

In [5]:

import tensorflow as tf
import pathlib

# Data splitting (entrenamiento y validación)
all_images_in_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  directorio_dataset,
  #validation_split=0.2,
  color_mode='rgb',
  label_mode='int',
  #subset="training",
  #seed=123,
  shuffle=False,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 10000 files belonging to 2 classes.


In [6]:
class_names = all_images_in_dataset.class_names
num_classes = len(class_names)  # get the number of the classes
print(f"Número de clases: {num_classes}")
print(f"Los nombres de las clases son: {class_names}")

Número de clases: 2
Los nombres de las clases son: ['cat', 'cat_c_defocus_blur']


In [7]:
for images, labels0 in all_images_in_dataset.take(1):  # only take first element of dataset
    numpy_all_images = images.numpy()  # convertir el tensor de imágenes en un arreglo de numpy y ahora contiene las imágenes del conjunto de datos en formato de numpy.
    numpy_all_labels = labels0.numpy()  # convierte el tensor de etiquetas en un arreglo de numpy y ahora contiene las etiquetas del conjunto de datos en formato de numpy

numpy_all_images /= 255.0  # Scale the features to the [0, 1] range
print(numpy_all_images.shape)

(10000, 32, 32, 3)


In [8]:
# Obtener todos los nombres de archivo en el dataset
filenames = [str(f) for f in directorio_dataset.glob('*/*.png')]

# Crear un mapeo de índices a rutas completas de archivo  
filenames_dict = {i: str(f) for i, f in enumerate(directorio_dataset.glob('*/*.png'))} 

In [12]:
images_cat = numpy_all_images[numpy_all_labels == 0][:5000]  # Clase 0: "cat"
images_cat_c_defocus_blur = numpy_all_images[numpy_all_labels == 1][:5000]  # Clase 1: "cat_c_defocus_blur"
images_cat = images_cat.astype(np.float64)
images_cat_c_defocus_blur = images_cat_c_defocus_blur.astype(np.float64)

# Inicializamos una lista para almacenar los resultados del CSG
csg_results = []

In [13]:
# Iterar sobre cada muestra de la clase "cat_c_defocus_blur"
for i in range(images_cat_c_defocus_blur.shape[0]):
    # Seleccionar una única muestra de la clase "cat_c_defocus_blur"
    single_image_cat_c_defocus_blur = images_cat_c_defocus_blur[i:i+1]

    # Combinar todas las muestras de "cat" con una muestra de "cat_c_defocus_blur"
    combined_images = np.concatenate((images_cat, single_image_cat_c_defocus_blur))

    # Crear etiquetas correspondientes (5000 para "cat" y 1 para "cat_c_defocus_blur")
    labels_cat = np.zeros(images_cat.shape[0], dtype=int)
    labels_single_cat_c_defocus_blur = np.array([1], dtype=int)
    combined_labels = np.concatenate((labels_cat, labels_single_cat_c_defocus_blur))

    # Asegurarse de que los datos estén en el formato correcto para el estimador
    fully_dataset = combined_images.reshape((combined_images.shape[0], combined_images.shape[1]*combined_images.shape[2]*combined_images.shape[3]))
    fully_labels = combined_labels

    # Usar el estimador con los datos combinados de las dos clases
    estimator = CumulativeGradientEstimator(M_sample=1000, k_nearest=10)
    estimator.fit(data=fully_dataset, target=fully_labels)

    # Guardar el resultado del CSG de esta comparación
    csg = estimator.csg
    csg_results.append(csg)





In [15]:
# Asociar índices con valores de CSG
csg_results_with_index = [(csg[0], i) for i, csg in enumerate(csg_results)]

# Ordenar los resultados de CSG por valor en orden descendente y seleccionar los 10 más altos
csg_results_sorted = sorted(csg_results_with_index, key=lambda x: x[0], reverse=True)[:10]

# Imprimir los 10 valores más altos junto con el índice de la imagen y el nombre del archivo
print("\nLos 10 valores más altos de CSG y sus respectivos índices:")
for csg_value, index in csg_results_sorted:
    # Asegurarse de que solo se obtengan los archivos de 'cat_c_defocus_blur'
    filename = filenames_dict.get(index + len(images_cat), "Archivo no encontrado")
    print(f"Índice: {index}, Valor CSG: {csg_value}, Nombre del archivo: {filename}")


Los 10 valores más altos de CSG y sus respectivos índices:
Índice: 0, Valor CSG: 1.0, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\00000.png
Índice: 1, Valor CSG: 1.0, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\00008.png
Índice: 5, Valor CSG: 1.0, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\00063.png
Índice: 6, Valor CSG: 1.0, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\00068.png
Índice: 7, Valor CSG: 1.0, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\00077.png
Índice: 10, Valor CSG: 1.0, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\00103.png
Índice: 11, Valor CSG: 1.0, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\00106.png
Índice: 12, Valor CSG: 1.0, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\00115.png
Índice: 14, Valor CSG: 1.0, Nombre del archivo: ..\Datasets\CIFAR-10 dos 

In [16]:
# Asociar índices con valores de CSG
csg_results_with_index = [(csg[0], i) for i, csg in enumerate(csg_results)]

# Ordenar los resultados de CSG por valor en orden ascendente y seleccionar los 10 más bajos
csg_results_sorted = sorted(csg_results_with_index, key=lambda x: x[0])[:10]

# Imprimir los 10 valores más bajos junto con el índice de la imagen y el nombre del archivo
print("\nLos 10 valores más bajos de CSG y sus respectivos índices:")
for csg_value, index in csg_results_sorted:
    # Asegurarse de que solo se obtengan los archivos de 'cat_c_defocus_blur'
    filename = filenames_dict.get(index + len(images_cat), "Archivo no encontrado")
    print(f"Índice: {index}, Valor CSG: {csg_value}, Nombre del archivo: {filename}")



Los 10 valores más bajos de CSG y sus respectivos índices:
Índice: 4990, Valor CSG: 0.9891884086729094, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\49947.png
Índice: 3990, Valor CSG: 0.9905717151454363, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\39947.png
Índice: 2990, Valor CSG: 0.9914829659318637, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\29947.png
Índice: 1990, Valor CSG: 0.9920762286860582, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\19947.png
Índice: 990, Valor CSG: 0.9921765358414201, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\09947.png
Índice: 4185, Valor CSG: 0.9922768304914744, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\41883.png
Índice: 3387, Valor CSG: 0.992684842112469, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\33956.png
Índice: 4548, Valor CSG: 0.9927783350057653, Nombre del arch

In [18]:
# Definir el índice específico que deseas revisar
indice_especifico = 4990  # índice que desees verificar

# Verificar que el índice especificado esté dentro del rango de csg_results
if 0 <= indice_especifico < len(csg_results):
    # Obtener el valor de CSG en el índice específico
    csg_value = csg_results[indice_especifico][0]

    # Obtener el nombre del archivo correspondiente usando el diccionario
    filename = filenames_dict.get(indice_especifico + len(images_cat), "Archivo no encontrado")

    # Mostrar el valor de CSG y el nombre del archivo
    print(f"Índice: {indice_especifico}, Valor CSG: {csg_value}, Nombre del archivo: {filename}")
else:
    print("Índice fuera del rango de csg_results.")


Índice: 4990, Valor CSG: 0.9891884086729094, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\49947.png


In [19]:
# Nombre del archivo de salida
output_filename = "resultados_csg.txt"

# Abrir el archivo en modo de escritura
with open(output_filename, "w") as file:
    # Iterar sobre los resultados de csg y sus índices
    for index, csg_result in enumerate(csg_results):
        # Obtener el valor de CSG
        csg_value = csg_result[0]  # asumimos que csg_result es un array o lista con el valor de CSG en la primera posición
        
        # Obtener el nombre del archivo correspondiente usando el diccionario
        filename = filenames_dict.get(index + len(images_cat), "Archivo no encontrado")
        
        # Escribir en el archivo de texto
        file.write(f"Índice: {index}, Valor CSG: {csg_value}, Nombre del archivo: {filename}\n")

print(f"Resultados exportados a {output_filename}")


Resultados exportados a resultados_csg.txt


In [23]:
# Nombre del archivo de entrada
input_filename = "resultados_csg.txt"

# Crear un diccionario para almacenar los resultados
csg_dict = {}

# Leer el archivo y almacenar los resultados en el diccionario
with open(input_filename, "r") as file:
    for line in file:
        # Extraer el índice, el valor de CSG y el nombre del archivo de cada línea
        parts = line.strip().split(", ")
        index = int(parts[0].split(": ")[1])
        csg_value = float(parts[1].split(": ")[1])
        filename = parts[2].split(": ")[1]
        
        # Almacenar en el diccionario
        csg_dict[index] = (csg_value, filename)

# Definir el índice específico que deseas revisar
indice_especifico = 4990  # índice que desees verificar

# Verificar que el índice especificado esté en el diccionario
if indice_especifico in csg_dict:
    csg_value, filename = csg_dict[indice_especifico]
    print(f"Índice: {indice_especifico}, Valor CSG: {csg_value}, Nombre del archivo: {filename}")
else:
    print("Índice no encontrado en el archivo.")


Índice: 4990, Valor CSG: 0.9891884086729094, Nombre del archivo: ..\Datasets\CIFAR-10 dos clases\cat_c_defocus_blur\49947.png
